In [1]:
import pandas as pd
from collections import defaultdict
from collections import Counter

In [2]:
# 서울시 공공자전거 대여 이력 정보
bike = pd.read_csv('data/bike.csv', encoding='cp949')
bike.head()

,자전거번호,대여일시,대여 대여소번호,대여 대여소명,대여거치대,반납일시,반납대여소번호,반납대여소명,반납거치대,이용시간,이용거리
0,SPB-53035,2021-12-01 09:45:56,3,중랑센터,0,2021-12-01 09:58:57,646,장한평역 1번출구 (국민은행앞),0,13,0.00
1,SPB-40999,2021-12-05 16:02:31,3,중랑센터,0,2021-12-05 16:07:58,529,장한평역 8번 출구 앞,0,5,0.00
2,SPB-55934,2021-12-10 22:01:39,3,중랑센터,0,2021-12-10 22:40:27,1332,석계역 5번출구 건너편,0,38,8710.00
3,SPB-50913,2021-12-11 07:01:32,3,중랑센터,0,2021-12-11 07:11:11,540,군자역 7번출구 베스트샵 앞,0,9,0.00
4,SPB-31324,2021-12-15 16:02:04,3,중랑센터,0,2021-12-15 16:13:21,540,군자역 7번출구 베스트샵 앞,0,11,2161.63


In [3]:
# 공공자전거 대여소 정보
bike_station = pd.read_csv('data/bike_station.csv')
bike_station.tail()

,대여소 번호,보관소(대여소)명,자치구,상세주소,위도,경도,설치시기,거치대수(LCD),거치대수(QR),운영방식
2581,5070,마곡13단지힐스테이트마스터 후문(1320동 앞),강서구,서울시 강서구 마곡중앙1로 71 마곡13단지힐스테이트마스터 후문 인근,37.556763,126.818695,2021.11.11,NaN,15.0,QR
2582,5072,김포공항입구 교통섬,강서구,공항동 74-5,37.561546,126.807083,2021.9.28,NaN,10.0,QR
2583,5073,우장산역 헤링턴타워 앞,강서구,화곡동 1013-3,37.546619,126.836235,2021.10.20,NaN,6.0,QR
2584,5074,마곡동 주니어마곡점앞,강서구,강서구 마곡동 798-17,37.558235,126.825760,2021.11.19,NaN,10.0,QR
2585,5075,템퍼 목동점,강서구,강서구 등촌동 649-14,37.551884,126.863571,2021.11.19,NaN,10.0,QR


In [4]:
# 전처리_#1
bike_station_dict = defaultdict(list)

id = bike_station['대여소 번호'].tolist()
address = bike_station['자치구'].tolist()
lat = bike_station['위도'].tolist()
lon = bike_station['경도'].tolist()

for i in range(len(bike_station)):
  bike_station_dict[id[i]].append(address[i])
  bike_station_dict[id[i]].append(lat[i])
  bike_station_dict[id[i]].append(lon[i])

In [10]:
# 전처리_#2
origination_name = bike['대여 대여소명'].tolist()
origination_list = bike['대여 대여소번호'].tolist()
destination_name = bike['반납대여소명'].tolist()
destination_list = bike['반납대여소번호'].tolist()

o_name = []
o_lat = []
o_lon = []
o_addr = []

d_name = []
d_lat = []
d_lon = []
d_addr = []

count = []

path_list = []

for i in range(len(bike)):
  if bike_station_dict.get(origination_list[i]) and bike_station_dict.get(destination_list[i]):
    if bike_station_dict.get(origination_list[i])[0] == '중구' or bike_station_dict.get(origination_list[i])[0] == '종로구':
      if bike_station_dict.get(destination_list[i])[0] == '중구' or bike_station_dict.get(destination_list[i])[0] == '종로구':
        if origination_list[i] != destination_list[i]:
          temp = []

          temp.append(origination_name[i])
          temp.append(bike_station_dict[origination_list[i]][0])
          temp.append(bike_station_dict[origination_list[i]][1])
          temp.append(bike_station_dict[origination_list[i]][2])
          temp.append(destination_name[i])
          temp.append(bike_station_dict[destination_list[i]][0])
          temp.append(bike_station_dict[destination_list[i]][1])
          temp.append(bike_station_dict[destination_list[i]][2])

          path_list.append(str(temp))

path_stat = Counter(path_list)

for key, value in path_stat.items():
  if value > 10:
    key = eval(key)
    o_name.append(key[0])
    o_addr.append(key[1])
    o_lat.append(key[2])
    o_lon.append(key[3])
    d_name.append(key[4])
    d_addr.append(key[5])
    d_lat.append(key[6])
    d_lon.append(key[7])
    count.append(value)

preprocessing_data = pd.DataFrame({
  'o_name': o_name,
  'o_addr': o_addr,
  'o_lat': o_lat,
  'o_lon': o_lon,
  'd_name': d_name,
  'd_addr': d_addr,
  'd_lat': d_lat,
  'd_lon': d_lon,
  'visited': count
})


In [11]:
preprocessing_data.to_csv('data/prerprocessing.csv', index=False, encoding='utf-8')
preprocessing_data.head(10)


,o_name,o_addr,o_lat,o_lon,d_name,d_addr,d_lat,d_lon,visited
0,정동사거리,중구,37.56805,126.969231,정동길입구,중구,37.565052,126.973404,67
1,정동사거리,중구,37.56805,126.969231,시청역(2호선) 9번출구 뒤,중구,37.563229,126.974838,36
2,정동사거리,중구,37.56805,126.969231,경희궁 자이 3단지,종로구,37.567490,126.965919,16
3,정동사거리,중구,37.56805,126.969231,종각역 5번출구,종로구,37.569836,126.982658,16
4,정동사거리,중구,37.56805,126.969231,부산은행,중구,37.566471,126.979256,25
5,정동사거리,중구,37.56805,126.969231,광화문역 6번출구 옆,종로구,37.569889,126.976456,24
6,정동사거리,중구,37.56805,126.969231,KG타워 앞,중구,37.563396,126.968941,13
7,정동사거리,중구,37.56805,126.969231,광화문 S타워 앞,종로구,37.569969,126.973938,49
8,정동사거리,중구,37.56805,126.969231,독립문역 사거리,종로구,37.572029,126.960785,17
9,정동사거리,중구,37.56805,126.969231,서울도시건축전시관,중구,37.567017,126.976837,40
